In [2]:
import tensorflow as tf
import time
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('mnist/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x12c122be0>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x13042c7f0>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x13042c7b8>)

# 定义超参数

In [6]:
n_classes=10
lr=1e-4
drop_out=0.5
hidden_size=1024
banch_size=32
epoch=5000
padding='SAME'
kernel_size=[5，5]
padding_size=[2,2]

# 定义PlaceHolder

In [7]:
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

## 第一层：卷积+池化

In [8]:
x_image = tf.reshape(x,[-1,28,28,1])
conv_1=tf.layers.conv2d(
    inputs=x_image,
    filters=32,
    kernel_size=kernel_size,
    padding=padding,
    activation=tf.nn.relu
)
pool_1=tf.layers.max_pooling2d(
    inputs=conv_1, 
    pool_size=padding_size, 
    strides=2,
    padding=padding
)

## 第二层：卷积+池化

In [9]:
conv_2=tf.layers.conv2d(
    inputs=x_image,
    filters=64,
    kernel_size=kernel_size,
    padding=padding,
    activation=tf.nn.relu
)
pool_2=tf.layers.max_pooling2d(
    inputs=conv_1, 
    pool_size=padding_size, 
    strides=2,
    padding=padding
)

## 第三层：全连接

In [11]:
fc1 = tf.layers.flatten(pool_2)#拍扁：降成一维
fc2 = tf.layers.dense(fc1, 1024 ,activation=tf.nn.relu)#全连接映射
fc2 = tf.layers.dropout(fc2, rate=keep_prob)
out= tf.layers.dense(fc2, n_classes)#全连接映射

## 梯度优化：
1. 构造损失函数
2. 构造梯度优化函数

In [13]:
cross_loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=out, labels=y)
loss = tf.reduce_mean(cross_loss)
train_step = tf.train.AdamOptimizer(lr).minimize(loss)
correct_prediction = tf.equal(tf.argmax(out,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

# 训练

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1,epoch+1):
        x_train,y_train = mnist.train.next_batch(64)
        _,l,acc=sess.run([train_step,loss,accuracy],feed_dict={x:x_train,y:y_train,keep_prob:drop_out})
        if i%100 == 0:
            print("Epoch:%d\t--\tLoss:%.3f\t--\tAcc:%.2f" % (i, l, acc))
        if i%1000 == 0:
            print(i)
            train_ac=sess.run([accuracy],feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
            print('测试集Acc:%.2f'.format(train_ac[0]))